In [1]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

## Python Version
import sys
print("Python: {0}".format(sys.version))

from ioUtils import saveJoblib, loadJoblib, saveFile, getFile
import urllib
from urllib.parse import quote

from discogsBase import discogs
from discogsUtils import discogsUtils
from collection import collections
from artist import artist
from timeUtils import clock, elapsed
from fsUtils import moveFile, setFile, setDir, setSubDir, isFile, isDir, mkDir
from fileUtils import getFileBasics, getBasename
from artists import artists
from albums import albums
from time import sleep
import time
from multiprocessing import Pool

import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

Python: 3.7.3 (default, Mar 27 2019, 16:54:48) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2020-01-15 17:44:22.063793


In [2]:
%load_ext autoreload
%autoreload
disc = discogs()
arts = artists(disc)
albs = albums(disc)
dutils = discogsUtils()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Saved Discog Directory /Volumes/Music/Discog is Available
Local Discog Directory /Users/tgadfort/Music/Discog is Available
/Volumes/Music/Discog/collections exists
/Volumes/Music/Discog/artists exists
/Volumes/Music/Discog/albums exists
/Volumes/Music/Discog/collections-db exists
/Volumes/Music/Discog/artists-db exists
/Volumes/Music/Discog/albums-db exists
/Volumes/Music/Discog/artists-db/metadata exists
/Volumes/Music/Discog/albums-db/metadata exists
/Volumes/Music/Discog/diagnostic exists
/Volumes/Music/Discog/db exists


# Collections

In [ ]:
%load_ext autoreload
%autoreload
#col = collections(disc)
#col.parseCollections(force=False)
#col.createCollectionDBs()

# Parse Artists (Parallel)

In [13]:
%load_ext autoreload
%autoreload

from artist import artist
from artists import artists
arts = artists(disc)

def parseArtists(modVal, force=False, doExtra=False):
    arts.parseArtistModValFiles(modVal, force=force)
    #arts.createArtistModValMetadata(modVal=modVal)
    #arts.createArtistAlbumModValMetadata(modVal=modVal)
    
def parseArtistsParallel(nProcs=3, force=False):
    pool = Pool(processes=nProcs)
    #result = pool.map_async(parseArtists, range(100)) 
    result = pool.map_async(parseArtists, [68, 62, 69, 70, 71])

    while not result.ready():
        if force is True:
            time.sleep(10)
        else:
            time.sleep(1)
    print("")
    return result.get()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
#arts.parseArtistModValFiles(27, force=True)
#arts.parseArtistModValFiles(50, force=True)

In [14]:
start,cmt=clock("Parsing All Artists")
start2,cmt2=clock("Parallel Artist Parsing")
parseArtistsParallel()
elapsed(start2, cmt2)
elapsed(start, cmt)

Current Time is Wed Jan 15, 2020 19:33:30 for Parsing All Artists
Current Time is Wed Jan 15, 2020 19:33:30 for Parallel Artist Parsing
Parsing Artist Files For ModVal 62
Parsing Artist Files For ModVal 68
Parsing Artist Files For ModVal 69
  ===> Found 6588 files. There are 95 new files to parse.
  ===> Found 6680 files, but there are no new files to parse so skipping.
Parsing Artist Files For ModVal 70
  ===> Found 6759 files. There are 118 new files to parse.
  ===> Found 6593 files, but there are no new files to parse so skipping.
Parsing Artist Files For ModVal 71
  ===> Found 6573 files, but there are no new files to parse so skipping.
Saving 95 new artist IDs to /Volumes/Music/Discog/artists-db/68-DB.p
Saving 118 new artist IDs to /Volumes/Music/Discog/artists-db/62-DB.p
  --> This file is 11.8MB.
Saving 6588 new artist IDs name data to /Volumes/Music/Discog/artists-db/metadata/68-Metadata.p
  --> This file is 385.5kB.
Saving 6588 new artist IDs media data to /Volumes/Music/Disc

# Albums

In [5]:
%load_ext autoreload
%autoreload
from album import album
from albums import albums
from discogsBase import discogs

disc = discogs()
albs = albums(disc)
#albs.parseAlbumModValFiles(modVal=27)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Saved Discog Directory /Volumes/Music/Discog is Available
Local Discog Directory /Users/tgadfort/Music/Discog is Available
/Volumes/Music/Discog/collections exists
/Volumes/Music/Discog/artists exists
/Volumes/Music/Discog/albums exists
/Volumes/Music/Discog/collections-db exists
/Volumes/Music/Discog/artists-db exists
/Volumes/Music/Discog/albums-db exists
/Volumes/Music/Discog/artists-db/metadata exists
/Volumes/Music/Discog/albums-db/metadata exists
/Volumes/Music/Discog/diagnostic exists
/Volumes/Music/Discog/db exists


In [7]:
def parseAlbums(modVal):
    albs.parseAlbumModValFiles(modVal=modVal)
    #albs.createAlbumModValMetadata(modVal)
    #albs.createAlbumModValMetadata(modVal=modVal)

def parseAlbumsParallel(modVals=None):
    if modVals is None:
        modVals = ["NAN"] + list(range(100))
        #modVals = range(100)
    nProcs = 2
    pool = Pool(processes=nProcs)
    result = pool.map_async(parseAlbums, modVals)
 
    start, cmt = clock("Processing {0} modVals using {1} processes".format(modVals, nProcs))
    print("Runing .",end="")
    while not result.ready():
        #print(".",end="")
        time.sleep(10)
    print("")
    
    elapsed(start, cmt)
    return result.get()

In [8]:
%load_ext autoreload
%autoreload
from album import album
from albums import albums
albs = albums(disc)

start,cmt=clock("Parsing All Albums")
start2,cmt2=clock("Parallel Album Parsing")
parseAlbumsParallel()
elapsed(start2, cmt2)
elapsed(start, cmt)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Current Time is Mon Jan 13, 2020 16:44:11 for Parsing All Albums
Current Time is Mon Jan 13, 2020 16:44:11 for Parallel Album Parsing
Current Time is Mon Jan 13, 2020 16:44:11 for Parsing Album Files For ModVal NAN
Current Time is Mon Jan 13, 2020 16:44:11 for Parsing Album Files For ModVal 12
  Looking for artist albums in /Volumes/Music/Discog/albums/NAN
  Looking for artist albums in /Volumes/Music/Discog/albums/12
Current Time is Mon Jan 13, 2020 16:44:11 for Processing ['NAN', 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99] modVals using 2 processes
Runing . 

  --> This file is 2.6MB.
Current Time is Mon Jan 13, 2020 17:11:56 for Done with Parsing Album Files For ModVal 15
Process [{0}] took 13.8 minutes.
Current Time is Mon Jan 13, 2020 17:11:56 for Parsing Album Files For ModVal 16
  Looking for artist albums in /Volumes/Music/Discog/albums/16
  Found 5263 artist directories
    Found 0   albums (0    ) in 0/5263	/Volumes/Music/Discog/albums/16/1000016
    Found 0   albums (10   ) in 100/5263	/Volumes/Music/Discog/albums/16/1082316
    Found 0   albums (46   ) in 500/5263	/Volumes/Music/Discog/albums/16/1360916
    Found 0   albums (79   ) in 1000/5263	/Volumes/Music/Discog/albums/16/170716
    Found 0   albums (99   ) in 1500/5263	/Volumes/Music/Discog/albums/16/2044516
    Found 0   albums (121  ) in 2000/5263	/Volumes/Music/Discog/albums/16/2438016
    Found 0   albums (153  ) in 2500/5263	/Volumes/Music/Discog/albums/16/2818516
    Found 0   albums (189  ) in 3000/5263	/Volumes/Music/Discog/albums/16/334416
    Found 0   albums (218  

  --> This file is 14.5MB.
Saving 4702 new artist IDs to /Volumes/Music/Discog/albums-db/metadata/1-ArtistMetadata.p
  --> This file is 1.1MB.
Saving 4702 new artist IDs to /Volumes/Music/Discog/albums-db/metadata/1-ArtistAlbums.p
  --> This file is 2.1MB.
Current Time is Mon Jan 13, 2020 17:23:55 for Done with Parsing Album Files For ModVal 1
Process [{0}] took 4.2 minutes.
Current Time is Mon Jan 13, 2020 17:23:56 for Parsing Album Files For ModVal 2
  Looking for artist albums in /Volumes/Music/Discog/albums/2
  --> This file is 18.1MB.
  Found 4424 artist directories
    Found 0   albums (0    ) in 0/4424	/Volumes/Music/Discog/albums/2/1000602
Saving 6705 new artist IDs to /Volumes/Music/Discog/albums-db/metadata/18-ArtistMetadata.p
  --> This file is 1.2MB.
Saving 6705 new artist IDs to /Volumes/Music/Discog/albums-db/metadata/18-ArtistAlbums.p
    Found 0   albums (16   ) in 100/4424	/Volumes/Music/Discog/albums/2/1105702
  --> This file is 2.8MB.
Current Time is Mon Jan 13, 2020

    Found 0   albums (69   ) in 1500/5283	/Volumes/Music/Discog/albums/4/2045704
  Found 3811 artist directories
    Found 0   albums (0    ) in 0/3811	/Volumes/Music/Discog/albums/21/100021
    Found 0   albums (12   ) in 100/3811	/Volumes/Music/Discog/albums/21/106321
    Found 0   albums (85   ) in 2000/5283	/Volumes/Music/Discog/albums/4/241304
    Found 0   albums (103  ) in 2500/5283	/Volumes/Music/Discog/albums/4/275604
    Found 0   albums (55   ) in 500/3811	/Volumes/Music/Discog/albums/21/1492221
    Found 0   albums (144  ) in 3000/5283	/Volumes/Music/Discog/albums/4/33004
    Found 0   albums (113  ) in 1000/3811	/Volumes/Music/Discog/albums/21/2107021
    Found 0   albums (174  ) in 3500/5283	/Volumes/Music/Discog/albums/4/393004
    Found 0   albums (163  ) in 1500/3811	/Volumes/Music/Discog/albums/21/276321
    Found 0   albums (215  ) in 4000/5283	/Volumes/Music/Discog/albums/4/4796604
    Found 1   albums (242  ) in 4500/5283	/Volumes/Music/Discog/albums/4/575704
    F

Process [{0}] took 6.0 minutes.
Current Time is Mon Jan 13, 2020 17:45:08 for Parsing Album Files For ModVal 7
  Looking for artist albums in /Volumes/Music/Discog/albums/7
    Found 0   albums (31   ) in 500/6061	/Volumes/Music/Discog/albums/23/1356223
    Found 0   albums (51   ) in 1000/6061	/Volumes/Music/Discog/albums/23/1667823
    Found 0   albums (69   ) in 1500/6061	/Volumes/Music/Discog/albums/23/2009623
  Found 4606 artist directories
    Found 0   albums (0    ) in 0/4606	/Volumes/Music/Discog/albums/7/1000507
    Found 0   albums (10   ) in 100/4606	/Volumes/Music/Discog/albums/7/1062007
    Found 0   albums (88   ) in 2000/6061	/Volumes/Music/Discog/albums/23/2380323
    Found 0   albums (38   ) in 500/4606	/Volumes/Music/Discog/albums/7/1404807
    Found 0   albums (107  ) in 2500/6061	/Volumes/Music/Discog/albums/23/27423
    Found 0   albums (77   ) in 1000/4606	/Volumes/Music/Discog/albums/7/1880007
    Found 0   albums (132  ) in 3000/6061	/Volumes/Music/Discog/album

...

  --> This file is 1.5MB.
Saving 6374 new artist IDs to /Volumes/Music/Discog/albums-db/metadata/79-ArtistAlbums.p
  --> This file is 3.8MB.
Current Time is Mon Jan 13, 2020 20:12:32 for Done with Parsing Album Files For ModVal 79
Process [{0}] took 5.1 minutes.
Current Time is Mon Jan 13, 2020 20:12:33 for Parsing Album Files For ModVal 80
  Looking for artist albums in /Volumes/Music/Discog/albums/80
  Found 6730 artist directories
    Found 0   albums (0    ) in 0/6730	/Volumes/Music/Discog/albums/75/1000575
    Found 0   albums (180  ) in 100/6730	/Volumes/Music/Discog/albums/75/1065875
  Found 6341 artist directories
    Found 0   albums (0    ) in 0/6341	/Volumes/Music/Discog/albums/80/1000180
    Found 0   albums (3    ) in 100/6341	/Volumes/Music/Discog/albums/80/1063880
    Found 0   albums (32   ) in 500/6341	/Volumes/Music/Discog/albums/80/1363380
    Found 0   albums (71   ) in 1000/6341	/Volumes/Music/Discog/albums/80/167080
    Found 1   albums (93   ) in 1500/6341	/Volum

    Found 0   albums (5493 ) in 4500/6775	/Volumes/Music/Discog/albums/82/439382
  --> This file is 18.0MB.
Saving 6711 new artist IDs to /Volumes/Music/Discog/albums-db/metadata/76-ArtistMetadata.p
  --> This file is 1.3MB.
Saving 6711 new artist IDs to /Volumes/Music/Discog/albums-db/metadata/76-ArtistAlbums.p
  --> This file is 2.9MB.
Current Time is Mon Jan 13, 2020 20:35:18 for Done with Parsing Album Files For ModVal 76
Process [{0}] took 5.2 minutes.
Current Time is Mon Jan 13, 2020 20:35:18 for Parsing Album Files For ModVal 90
  Looking for artist albums in /Volumes/Music/Discog/albums/90
    Found 0   albums (6002 ) in 5000/6775	/Volumes/Music/Discog/albums/82/493582
  Found 6211 artist directories
    Found 0   albums (0    ) in 0/6211	/Volumes/Music/Discog/albums/90/1000590
    Found 0   albums (8    ) in 100/6211	/Volumes/Music/Discog/albums/90/1076190
    Found 0   albums (40   ) in 500/6211	/Volumes/Music/Discog/albums/90/135890
    Found 0   albums (74   ) in 1000/6211	

    Found 1   albums (2842 ) in 4000/6786	/Volumes/Music/Discog/albums/83/3850383
    Found 0   albums (100  ) in 1500/4727	/Volumes/Music/Discog/albums/93/2229893
    Found 0   albums (152  ) in 2000/4727	/Volumes/Music/Discog/albums/93/2792193
    Found 1   albums (3159 ) in 4500/6786	/Volumes/Music/Discog/albums/83/4439783
    Found 1   albums (209  ) in 2500/4727	/Volumes/Music/Discog/albums/93/3421193
    Found 0   albums (243  ) in 3000/4727	/Volumes/Music/Discog/albums/93/4168293
    Found 0   albums (280  ) in 3500/4727	/Volumes/Music/Discog/albums/93/496893
    Found 0   albums (3456 ) in 5000/6786	/Volumes/Music/Discog/albums/83/4975283
    Found 0   albums (311  ) in 4000/4727	/Volumes/Music/Discog/albums/93/601193
    Found 0   albums (365  ) in 4500/4727	/Volumes/Music/Discog/albums/93/828993
    Found 0   albums (3706 ) in 5500/6786	/Volumes/Music/Discog/albums/83/5421383
    Found 0   albums (402  ) in 4726/4727	/Volumes/Music/Discog/albums/93/99793
Saving 402 (new) / 40

  Looking for artist albums in /Volumes/Music/Discog/albums/96
    Found 0   albums (106  ) in 2000/6326	/Volumes/Music/Discog/albums/85/2406185
    Found 0   albums (119  ) in 2500/6326	/Volumes/Music/Discog/albums/85/280485
    Found 0   albums (147  ) in 3000/6326	/Volumes/Music/Discog/albums/85/3245485
    Found 0   albums (185  ) in 3500/6326	/Volumes/Music/Discog/albums/85/3654785
  Found 6634 artist directories
    Found 0   albums (0    ) in 0/6634	/Volumes/Music/Discog/albums/96/1000596
    Found 0   albums (11   ) in 100/6634	/Volumes/Music/Discog/albums/96/1062396
    Found 0   albums (225  ) in 4000/6326	/Volumes/Music/Discog/albums/85/4173185
    Found 0   albums (43   ) in 500/6634	/Volumes/Music/Discog/albums/96/1337796
    Found 1   albums (248  ) in 4500/6326	/Volumes/Music/Discog/albums/85/4810385
    Found 0   albums (273  ) in 5000/6326	/Volumes/Music/Discog/albums/85/5281585
    Found 0   albums (79   ) in 1000/6634	/Volumes/Music/Discog/albums/96/1633496
    Found

    Found 0   albums (234  ) in 3500/5683	/Volumes/Music/Discog/albums/98/4027298
  --> This file is 10.9MB.
Saving 3743 new artist IDs to /Volumes/Music/Discog/albums-db/metadata/87-ArtistMetadata.p
    Found 0   albums (264  ) in 4000/5683	/Volumes/Music/Discog/albums/98/4637598
  --> This file is 879.0kB.
Saving 3743 new artist IDs to /Volumes/Music/Discog/albums-db/metadata/87-ArtistAlbums.p
  --> This file is 1.5MB.
Current Time is Mon Jan 13, 2020 21:08:19 for Done with Parsing Album Files For ModVal 87
Process [{0}] took 3.3 minutes.
Current Time is Mon Jan 13, 2020 21:08:19 for Parsing Album Files For ModVal 88
  Looking for artist albums in /Volumes/Music/Discog/albums/88
    Found 0   albums (287  ) in 4500/5683	/Volumes/Music/Discog/albums/98/5253298
    Found 0   albums (319  ) in 5000/5683	/Volumes/Music/Discog/albums/98/6407498
    Found 0   albums (367  ) in 5500/5683	/Volumes/Music/Discog/albums/98/879698
    Found 0   albums (396  ) in 5682/5683	/Volumes/Music/Discog/a